# Sand pit

When I worked on AoC 2015, I noticed I can do better than that. I had no idea what to expect, and my approach was horrible. After reading the assignment, I had to search the internet regarding certain features in python. However, I did not want to just sit down and read all the python documentations from the start to the end. So I got some important highlight of the language to focus on. So, I decided to get some hands on training on the toolset that Peter Norvig <a href='https://nbviewer.org/github/norvig/pytudes/blob/main/ipynb/Advent-2021.ipynb'>used</a>. After, I will solve the puzzles, then compare mine with his and see how horrible mine is.

In [2]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import matplotlib.pyplot as plt
import re

## annotations

<code>annotations</code> delays evaluation of function annotations. Without the future import, NameError is raised below. However, after the import, it works without any issue.

In [2]:
class A:
    def a(self) -> A:
        return

## Collections

### Counter 

Counter can be used to count things.

There are many ways to initiate the Counter.

In [3]:
Counter()

Counter()

In [4]:
Counter('Darkness')

Counter({'D': 1, 'a': 1, 'r': 1, 'k': 1, 'n': 1, 'e': 1, 's': 2})

In [5]:
Counter({'bacon': 4, 'eggs': 30})

Counter({'bacon': 4, 'eggs': 30})

In [6]:
c = Counter(onions=2, cucumber=19, potato=11, sweet_potato=38)
c

Counter({'onions': 2, 'cucumber': 19, 'potato': 11, 'sweet_potato': 38})

Counts can be retrieved by using keys. If it does not exist in the Counter, count is 0.

In [7]:
c['onions'], c['zombie']

(2, 0)

Counts can be changed. It can be 0 or negative. 

In [8]:
c['sweet_potato'] = 0
'sweet_potato' in c

True

In [9]:
c['sweet_potato'] = -100
'sweet_potato' in c

True

Elements in the Counts can be deleted with del. 

In [10]:
del c['sweet_potato']
'sweet_potato' in c

False

Counter has help methods. <code>elements</code> returns an iterator that generates all the elements.

In [11]:
list(c.elements())

['onions',
 'onions',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'cucumber',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato',
 'potato']

Most common items from top to bottom.

In [14]:
c.most_common()

[('cucumber', 19), ('potato', 11), ('onions', 2)]

In [13]:
c.most_common(1)

[('cucumber', 19)]

Counter can subtract others, such as mappings or iterables. It is not an error if some elemtns are missing.

In [17]:
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(a=1, b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 3, 'b': 0, 'c': -3, 'd': -6})

In [18]:
# Missing subtract element
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 4, 'b': 0, 'c': -3, 'd': -6})

In [20]:
# Missing original element
c = Counter(a=4, b=2, c=0)
d = Counter(a=1, b=2, c=3, d=4)
c.subtract(d)
c

Counter({'a': 3, 'b': 0, 'c': -3, 'd': -4})

In [27]:
# Subtracting dictionary
c = Counter(a=4, b=2, c=0, d=-2)
d = {'a':3}
c.subtract(d)
c

Counter({'a': 1, 'b': 2, 'c': 0, 'd': -2})

Counter can also add, but it is called update. Instead of replacing, it simply adds counts.

In [28]:
c.update('abbccc')
c

Counter({'a': 2, 'b': 4, 'c': 3, 'd': -2})

Mathematical operations can be used with Counter objects. Negative counts are removed.

In [45]:
c = Counter(a=2, b=-4, c=0, d=10)
d = Counter(a=-1, b=2, c=3, d=17)

In [47]:
# Add
c + d

Counter({'a': 1, 'c': 3, 'd': 27})

In [48]:
# Subtract
c - d

Counter({'a': 3})

In [49]:
# Intersection
c & d

Counter({'d': 10})

In [50]:
# Union
c | d

Counter({'a': 2, 'b': 2, 'c': 3, 'd': 17})

Unary operator + gets rid of negative and zero elements, and - gets rid of positive and zero elements.

In [33]:
c = Counter(a=2, b=-4, c=0)
+c

Counter({'a': 2})

In [34]:
-c

Counter({'b': 4})

### Defaultdict

Instead of using dictionary's <code>setdefault</code>, </code>defaultdict</code> can achieve higher efficiency and more flexibility. Here, it uses a list for the default value. 

In [38]:
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 11)]
d = defaultdict(list)
d

defaultdict(list, {})

In [16]:
for k, v in l:
    d[k].append(v)

sorted(d.items())

[('bacon', [3, 11]), ('egg', [10]), ('oj', [1, 3])]

Same thing could have achieved using plain dictionary's <code>setdefault</code>.

In [18]:
d2 = {}
for k, v in l:
    d2.setdefault(k, []).append(v)
sorted(d2.items())

[('bacon', [3, 11]), ('egg', [10]), ('oj', [1, 3])]

<code>defaultdict</code> can also use other type for the default value, such as integer. If integer is used, it can be used as a counter.

In [41]:
l = ['bacon', 'egg', 'egg', 'oj', 'bacon', 'bacon', 'bacon', 'bacon']
d = defaultdict(int)
for k in l:
    d[k] += 1
sorted(d.items())

[('bacon', 5), ('egg', 2), ('oj', 1)]

If I need a counter, it is better to use Counter().

In [33]:
d2 = Counter(l)
d2

Counter({'bacon': 5, 'egg': 2, 'oj': 1})

In [35]:
list(d2.elements())

['bacon', 'bacon', 'bacon', 'bacon', 'bacon', 'egg', 'egg', 'oj']

In [36]:
d2.most_common(2)

[('bacon', 5), ('egg', 2)]

Set can also be used as a default type.

In [43]:
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 3)]
d = defaultdict(set)
for k, v in l:
    d[k].add(v)
sorted(d.items())

[('bacon', {3}), ('egg', {10}), ('oj', {1, 3})]

Default value does not have to be set by the type. By passing a value with lambda, any constant value can be set to default.

In [46]:
def constant_factory(value):
    return lambda: value
d = defaultdict(constant_factory('<missing>'))
d.update(name='John', action='ran')
'%(name)s %(action)s to %(object)s' % d

'John ran to <missing>'

In [48]:
d2 = defaultdict(constant_factory(['Must Have!']))
l = [('bacon', 3), ('egg', 10), ('oj', 1), ('oj', 3), ('bacon', 11)]
for k,v in l:
    d2[k].append(v)
sorted(d2.items())

[('bacon', ['Must Have!', 3, 10, 1, 3, 11]),
 ('egg', ['Must Have!', 3, 10, 1, 3, 11]),
 ('oj', ['Must Have!', 3, 10, 1, 3, 11])]